In [5]:
import os, glob, random, itertools, calendar, datetime
from tqdm import tqdm
import missingno as mn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import pymannkendall as mk
import statsmodels.api as sm
import seaborn as sns
import matplotlib.dates as mdates 
from calendar import month_name
month_lookup = list(month_name)
from datetime import datetime

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

path = 'C:/Users/Alexandra/Desktop/WMO 2025/01 Simulations/Palestine/'
os.chdir(path)

def New_method_filter(df, percent = 80, start = '2005-01-01', end = '2024-12-31', output_excel_path='Rejected_Wells.xlsx' ):   
    
    """
    Implement an updated methodology for selecting data for future trend analysis, as described in the IGRAC report.

    Parameters:
    - df (DataFrame): DataFrame with time series.
    - percent (int): Selected threshold to filter data (default: 90).
    - start (str, '%Y-%m-%d'): Initial date for analysis (default: '2001-01-01').
    - end (str, '%Y-%m-%d'): Final date for analysis (default: '2020-12-31').

    Output:
    - Accepted (DataFrame): Data above the chosen threshold between the selected dates.
    - Rejected (DataFrame): Data below the chosen threshold between the selected dates.
    """

    
    start = datetime.strptime(start , '%Y-%m-%d')
    end = datetime.strptime(end , '%Y-%m-%d')
    
    df = df[(df.index >= start) & (df.index <= end)]
    
    Total_wells = len(list(df))
    
    Total_years_period = (len(df)/12)
    
    Total_years = (len(df)/12) * (percent*0.01)
    
    

    print(f"The chosen period covers {Total_years_period} years between {start:%Y-%m-%d} and {end:%Y-%m-%d}")
    print(f"With the {percent}% condition this period will accept {np.ceil(Total_years)}({round(Total_years,1)}) of {Total_years_period}")
    
    
    Final_df = []
    Rest_df = []
    
    rejected_wells_info = []

    for m, n in tqdm(enumerate(list(df))):


        Time_serie = df.iloc[:,m].reset_index()
        Time_serie_MC = Time_serie.groupby([Time_serie['Date'].dt.month_name()], sort=False).count()
        Meses_mas_datos = Time_serie_MC.drop(columns = 'Date').sort_values(by = n, ascending=False).index


        Month_number = []
        for h in Meses_mas_datos:
            Month_number.append(list(calendar.month_name).index(h))


        Month_complete = []
        Rest = [df.iloc[:,m]]

        for i,j in enumerate(Month_number) :

            #print(i,j)
            comp_dates = pd.DataFrame(pd.date_range(start=start+ pd.DateOffset(months=-1),\
                                                    end= end+ pd.DateOffset(months=+1), freq='M'), \
                                      columns=['Date'])
            Test1 = comp_dates.merge(Rest[i].reset_index(), how='left', on='Date')
            Rest[i] = pd.Series(Test1.iloc[:,1].values, index= Test1.iloc[:,0].values,\
                                name = Test1.iloc[:,1].name)
            Rest[i].index.name = 'Date'


            Remaining_1 = Rest[i][Rest[i].index.month != j]

            Sel_period = Rest[i][(Rest[i].index >= start) & (Rest[i].index <= end)]
            Month_sel = Sel_period[Sel_period.index.month == j ]


            Month_sel_nan = Month_sel[Month_sel.isnull()]
            #print(Month_sel_nan)

            #Estos son los seleccionados que no son NaN

            Month_sel_nonan = Month_sel.dropna()

            if len(Month_sel_nonan) == Total_years_period:

                Month_complete.append(Month_sel_nonan)
                Rest.append(Remaining_1)

            elif len(Month_sel_nonan) < Total_years_period:
                #print(i,j)



                Valores_reemplazo = [Month_sel_nonan]

                for k,h in enumerate(Month_sel_nan):


                    if  ~np.isnan(Remaining_1[Month_sel_nan.index[k] + pd.DateOffset(months=-1)+ pd.offsets.MonthEnd(n=0)]) & \
                        ~np.isnan(Remaining_1[Month_sel_nan.index[k] + pd.DateOffset(months=+1)+ pd.offsets.MonthEnd(n=0)]) == True:

                        Month_sel_nan[k] = np.mean([Remaining_1[Month_sel_nan.index[k] + pd.DateOffset(months=-1)+ pd.offsets.MonthEnd(n=0)]\
                                                    , Remaining_1[Month_sel_nan.index[k] + pd.DateOffset(months=+1)+ pd.offsets.MonthEnd(n=0)]])
        

                    elif  ~np.isnan(Remaining_1[Month_sel_nan.index[k] + pd.DateOffset(months=-1)+ pd.offsets.MonthEnd(n=0)]) == True:


                        Value = Remaining_1.loc[[Month_sel_nan.index[k] + pd.DateOffset(months=-1)+ pd.offsets.MonthEnd(n=0)]]

                        Valores_reemplazo.append(Value)


                        Remaining_1 = Remaining_1.drop(Month_sel_nan.index[k] + pd.DateOffset(months=-1)+ pd.offsets.MonthEnd(n=0))


                    elif  ~np.isnan(Remaining_1[Month_sel_nan.index[k] + pd.DateOffset(months=+1)+ pd.offsets.MonthEnd(n=0)]) == True:


                        Value = Remaining_1.loc[[Month_sel_nan.index[k] + pd.DateOffset(months=+1)+ pd.offsets.MonthEnd(n=0)]]


                        Valores_reemplazo.append(Value)


                        Remaining_1 = Remaining_1.drop(Month_sel_nan.index[k] + pd.DateOffset(months=+1)+ pd.offsets.MonthEnd(n=0))


                    elif np.isnan(Remaining_1[Month_sel_nan.index[k] + pd.DateOffset(months=-1)+ pd.offsets.MonthEnd(n=0)]) & \
                        np.isnan(Remaining_1[Month_sel_nan.index[k] + pd.DateOffset(months=+1)+ pd.offsets.MonthEnd(n=0)]) == True:

                        continue


                Paso1 = pd.concat(Valores_reemplazo).sort_index()



                if len(pd.concat([Paso1,Month_sel_nan.dropna()]).sort_index()) >= Total_years:



                    Month_complete.append(pd.concat([Paso1,Month_sel_nan.dropna()]).sort_index())
                    Rest.append(Remaining_1)

                else:

                    Rest.append(Rest[i])


        comp_dates1 = pd.DataFrame(pd.date_range(start=start, end=end, freq='M'), columns=['Date'])
        
        if len(Month_complete)>0 :

            Time_serie_NONAN = pd.concat(Month_complete).sort_index().reset_index()
            Final_Time_serie = comp_dates1.merge(Time_serie_NONAN, how='left', on='Date').set_index('Date')
            Rest_Time_serie = comp_dates1.merge(Rest[-1], how='left', on='Date').set_index('Date')
            Final_df.append(Final_Time_serie)
            Rest_df.append(Rest_Time_serie)
        else:

            Rest_Time_serie = comp_dates1.merge(Rest[-1], how='left', on='Date').set_index('Date')            
            Rest_df.append(Rest_Time_serie) 
            
            # Collect information about rejected wells
            rejected_wells_info.append({'Well': n, 'Reason': 'Insufficient Data'})

    if len(Final_df) >0:
        
        print(f"{len(Final_df)} well(s) accepted out of {Total_wells}")
        Final_accepted = pd.concat(Final_df, axis = 1)
        Not_accepted = pd.concat(Rest_df, axis = 1)
    else: 
        
        print(f"0 wells accepted out of {Total_wells}")
        Final_accepted = pd.Dataframe()
        Not_accepted = pd.concat(Rest_df, axis = 1)
    # Save rejected wells information to Excel
    rejected_wells_df = pd.DataFrame(rejected_wells_info)
    rejected_wells_df.to_excel(output_excel_path, index=False)

    
    return Final_accepted, Not_accepted

def New_method_filter2(df, percent = 80, start = '2005-01-01', end = '2024-12-31', output_excel_path='Rejected_Wells.xlsx' ):   
    
    """
    Implement an updated methodology for selecting data for future trend analysis, as described in the IGRAC report.

    Parameters:
    - df (DataFrame): DataFrame with time series.
    - percent (int): Selected threshold to filter data (default: 90).
    - start (str, '%Y-%m-%d'): Initial date for analysis (default: '2001-01-01').
    - end (str, '%Y-%m-%d'): Final date for analysis (default: '2020-12-31').

    Output:
    - Accepted (DataFrame): Data above the chosen threshold between the selected dates.
    - Rejected (DataFrame): Data below the chosen threshold between the selected dates.
    """

    
    start = datetime.strptime(start , '%Y-%m-%d')
    end = datetime.strptime(end , '%Y-%m-%d')
    
    df = df[(df.index >= start) & (df.index <= end)]
    
    Total_wells = len(list(df))
    
    Total_years_period = (len(df)/12)
    
    Total_years = (len(df)/12) * (percent*0.01)
    
    

    print(f"The chosen period covers {Total_years_period} years between {start:%Y-%m-%d} and {end:%Y-%m-%d}")
    print(f"With the {percent}% condition this period will accept {np.ceil(Total_years)}({round(Total_years,1)}) of {Total_years_period}")
    
    
    Final_df = []
    Rest_df = []
    
    rejected_wells_info = []

    for m, n in tqdm(enumerate(list(df))):


        Time_serie = df.iloc[:,m].reset_index()
        Time_serie_MC = Time_serie.groupby([Time_serie['Date'].dt.month_name()], sort=False).count()
        Meses_mas_datos = Time_serie_MC.drop(columns = 'Date').sort_values(by = n, ascending=False).index


        Month_number = []
        for h in Meses_mas_datos:
            Month_number.append(list(calendar.month_name).index(h))


        Month_complete = []
        Rest = [df.iloc[:,m]]
        used_values = set()

        for i,j in enumerate(Month_number) :
            
            #print(i,j)
            comp_dates = pd.DataFrame(pd.date_range(start=start+ pd.DateOffset(months=-1),\
                                                    end= end+ pd.DateOffset(months=+1), freq='M'), \
                                      columns=['Date'])
            Test1 = comp_dates.merge(Rest[i].reset_index(), how='left', on='Date')
            Rest[i] = pd.Series(Test1.iloc[:,1].values, index= Test1.iloc[:,0].values,\
                                name = Test1.iloc[:,1].name)
            Rest[i].index.name = 'Date'


            Remaining_1 = Rest[i][Rest[i].index.month != j]

            Sel_period = Rest[i][(Rest[i].index >= start) & (Rest[i].index <= end)]
            Month_sel = Sel_period[Sel_period.index.month == j ]


            Month_sel_nan = Month_sel[Month_sel.isnull()]
            #print(Month_sel_nan)

            #Estos son los seleccionados que no son NaN

            Month_sel_nonan = Month_sel.dropna()

            if len(Month_sel_nonan) == Total_years_period:

                Month_complete.append(Month_sel_nonan)
                Rest.append(Remaining_1)

            elif len(Month_sel_nonan) < Total_years_period:
                #print(i,j)



                Valores_reemplazo = [Month_sel_nonan]

                for k, h in enumerate(Month_sel_nan):
                    prev_month = Month_sel_nan.index[k] + pd.DateOffset(months=-1) + pd.offsets.MonthEnd(n=0)
                    next_month = Month_sel_nan.index[k] + pd.DateOffset(months=+1) + pd.offsets.MonthEnd(n=0)

                    if ~np.isnan(Remaining_1[prev_month]) and ~np.isnan(Remaining_1[next_month]):
                        # If there are valid values in the previous and next months, calculate the mean and replace the NaN value
                        Month_sel_nan[k] = np.mean([Remaining_1[prev_month], Remaining_1[next_month]])

                        # Update the values in the Remaining_1 dataframe
                        Remaining_1.loc[Month_sel_nan.index[k]] = np.mean([Remaining_1[prev_month], Remaining_1[next_month]])
                      
    
                    elif ~np.isnan(Remaining_1[prev_month]):
                        # If there's a valid value in the previous month, replace the NaN value with it
                        Month_sel_nan[k] = Remaining_1[prev_month]
                        
                        # Delete corresponding value
                        Remaining_1[prev_month] = np.nan

                        # Update the values in the Remaining_1 dataframe
                        Remaining_1.loc[Month_sel_nan.index[k]] = Remaining_1[prev_month]
                     

                    elif ~np.isnan(Remaining_1[next_month]) and next_month not in used_values:
                        # If there's a valid value in the next month, replace the NaN value with it
                        Month_sel_nan[k] = Remaining_1[next_month]
                        
                        Remaining_1[next_month] = np.nan

                        # Update the values in the Remaining_1 dataframe
                        Remaining_1.loc[Month_sel_nan.index[k]] = Remaining_1[next_month]
                     


                    elif np.isnan(Remaining_1[Month_sel_nan.index[k] + pd.DateOffset(months=-1)+ pd.offsets.MonthEnd(n=0)]) & \
                        np.isnan(Remaining_1[Month_sel_nan.index[k] + pd.DateOffset(months=+1)+ pd.offsets.MonthEnd(n=0)]) == True:

                        continue


                Paso1 = pd.concat(Valores_reemplazo).sort_index()



                if len(pd.concat([Paso1,Month_sel_nan.dropna()]).sort_index()) >= Total_years:



                    Month_complete.append(pd.concat([Paso1,Month_sel_nan.dropna()]).sort_index())
                    Rest.append(Remaining_1)

                else:

                    Rest.append(Rest[i])


        comp_dates1 = pd.DataFrame(pd.date_range(start=start, end=end, freq='M'), columns=['Date'])
        
        if len(Month_complete)>0 :

            Time_serie_NONAN = pd.concat(Month_complete).sort_index().reset_index()
            Final_Time_serie = comp_dates1.merge(Time_serie_NONAN, how='left', on='Date').set_index('Date')
            Rest_Time_serie = comp_dates1.merge(Rest[-1], how='left', on='Date').set_index('Date')
            Final_df.append(Final_Time_serie)
            Rest_df.append(Rest_Time_serie)
        else:

            Rest_Time_serie = comp_dates1.merge(Rest[-1], how='left', on='Date').set_index('Date')            
            Rest_df.append(Rest_Time_serie) 
            
            # Collect information about rejected wells
            rejected_wells_info.append({'Well': n, 'Reason': 'Insufficient Data'})

    if len(Final_df) >0:
        
        print(f"{len(Final_df)} well(s) accepted out of {Total_wells}")
        Final_accepted = pd.concat(Final_df, axis = 1)
        Not_accepted = pd.concat(Rest_df, axis = 1)
    else: 
        
        print(f"0 wells accepted out of {Total_wells}")
        Final_accepted = pd.Dataframe()
        Not_accepted = pd.concat(Rest_df, axis = 1)
    # Save rejected wells information to Excel
    rejected_wells_df = pd.DataFrame(rejected_wells_info)
    rejected_wells_df.to_excel(output_excel_path, index=False)

    
    return Final_accepted, Not_accepted



# Now you can use the DataFrame 'df' as needed
# Read data from Excel file
df = pd.read_csv('Palestine.csv', encoding='utf-8', parse_dates=['Date'], dayfirst=True)
#in Case of depth :
df['level'] = df['level'] * -1

# Set the date to the last day of the month
df['Date'] = df['Date'] + pd.offsets.MonthEnd(0)

# Group by ID, last day of the month, and calculate the average level
result = df.groupby(['Date', 'site']).agg({'level': 'mean'}).reset_index()

# Step 4: Pivot the DataFrame to create a 3D structure
df_3d = result.pivot_table(index='Date', columns='site', values='level', aggfunc='first')

df_3d.to_csv('Mean_Pivot_table.csv')
Data = pd.read_csv('Mean_Pivot_table.csv',parse_dates = True, index_col = 'Date')

#Removing time series with no data
#a = df_3d.describe().T 
#b = a[a['count'] != 0]
#Data = Data[list(b.T)]

#Apply the filter Method
Accepted_90, Rejected_90 = New_method_filter(Data)
Accepted_90_percentile, Rejected_90_percentile = New_method_filter2(Data)

Accepted_90_percentile.to_csv('Accepted_data_percentile.csv')
Accepted_90.to_csv('Accepted_data.csv')

def Test_plot_2(df1, df2, savefig=False, output_folder='comparison_plots'):
    """
    Generate a plot to visualize and compare data acceptance and data rejection.

    Parameters:
    - df1 (DataFrame): Accepted data DataFrame.
    - df2 (DataFrame): Rejected data DataFrame.
    - savefig (bool): If True, save the generated plot as an image file (default: False).
    - output_folder (str): Folder to save individual well plots (default: 'comparison_plots').

    Output:
    - Matplotlib Figure: Plot visualizing and comparing data acceptance and rejection.
    """
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for well in tqdm(list(df1.columns)):
        fig, ax = plt.subplots(1, 1, figsize=(30, 10), sharex=True)

        ax.set_title(well, fontsize=12,color = 'white')
        ax.plot(df1.index, df1[well], markersize=2, marker="o", color='darkblue', linestyle='', label='Accepted Value')
        ax.plot(df2.index, df2[well], markersize=2, marker="o", color=(0.7, 0.7, 1.0), linestyle='', label='Rejected Value')
        
        ax.set_xlim(df1.index[0], df1.index[-1])
        ax.grid()

        loc = mdates.YearLocator(1)
        ax.xaxis.set_major_locator(loc)
        fmt = mdates.DateFormatter('%Y')
        ax.xaxis.set_major_formatter(fmt)

        ax.figure.autofmt_xdate(rotation=45, ha='center')
        ax.legend()
        
        # Set the axis labels
        ax.set_xlabel('Date', color='white')  # X-axis label
        ax.set_ylabel('Depth (m)', color='white')  # Y-axis label
        # 'GWL (m a.m.s.l)' for GWL above mean sea level
        # 'Depth (m)' for GW depth
        
        # Set the axis tick labels' color to white
        for label in ax.get_xticklabels():
            label.set_color('white')
        for label in ax.get_yticklabels():
            label.set_color('white')
            tick_text = label.get_text()


        if savefig:
            plot_filename = os.path.join(output_folder, f'{well.replace("/", "_").replace(":", "_").replace("(", "").replace(")", "").strip()}.png')  # Replace '/' with '_'
            plt.savefig(plot_filename, bbox_inches='tight')
            plt.close()
    return


Test_plot_2(Accepted_90,Rejected_90, savefig=True, output_folder='comparison_plots')

df = Accepted_90_percentile
df = Accepted_90

# Directory to save the Excel files
output_directory = 'C:/Users/Alexandra/Desktop/WMO 2025/01 Simulations/Palestine/Palestine 60 20 years/output'
os.makedirs(output_directory, exist_ok=True)

# Create a dictionary to store percentiles, last year, and number of years considered for each well
well_info = {'Well': [], 'Percentile': [], 'Last_Year': [], 'Num_Years_Considered': [], 'Last_Year_Month_Count':[], 'Avg_Month_Count_Excluding_Last_Year': []}

# Iterate over each well column
for well in df.columns[0:]:
    # Filter data for the specific well
    well_data = df[[well]]
    
    # Calculate the number of months with data in the last year
    last_year_month_count = well_data.loc[well_data.index.year == well_data.index.year[-1]].notna().sum().iloc[0]

    # Calculate the number of months with data for all years except the last year
    all_years_data = well_data.loc[well_data.index.year != well_data.index.year[-1]]
    # Calculate the number of years considered for the well
    num_years_considered = all_years_data.index.year.nunique()
    month_count_excluding_last = all_years_data.notna().sum()
    avg_month_count_excluding_last = month_count_excluding_last / num_years_considered
    
    # Convert avg_month_count_excluding_last to numeric
    avg_month_count_excluding_last = pd.to_numeric(avg_month_count_excluding_last, errors='coerce')
  
    
    # Resample annually and calculate mean
    well_year_df = well_data.resample('Y').mean()

    # Drop NaN values
    well_year_df.dropna(inplace=True)

    if not well_year_df.empty:
        # Get the last available year for the well
        last_year = well_year_df.index.year[-1]
        # Calculate percent rank for the well in the last available year
        well_percentile = well_year_df[well].rank(pct=True).iloc[-1]
        # Get the number of years considered for the well
        num_years_considered = well_year_df.index.year.nunique()
    else:
        # If there's no data for the well, set percentiles, last year, and num years to NaN
        well_percentile = pd.NaT
        last_year = pd.NaT
        num_years_considered = pd.NaT

    # Append the well information to the dictionary
    well_info['Well'].append(well)
    well_info['Percentile'].append(well_percentile)
    well_info['Last_Year'].append(last_year)
    well_info['Num_Years_Considered'].append(num_years_considered)
    well_info['Last_Year_Month_Count'].append(last_year_month_count)
    well_info['Avg_Month_Count_Excluding_Last_Year'].append(float(avg_month_count_excluding_last))  # Append the average month count

# Create a DataFrame from the dictionary
result_df = pd.DataFrame(well_info)

# Save the reshaped DataFrame to an Excel file
result_df.to_excel(os.path.join(output_directory, 'percentiles_last_year_with_years.xlsx'), index=False)


The chosen period covers 20.0 years between 2005-01-01 and 2024-12-31
With the 60% condition this period will accept 12.0(12.0) of 20.0


290it [00:37,  7.81it/s]


71 well(s) accepted out of 290
The chosen period covers 20.0 years between 2005-01-01 and 2024-12-31
With the 60% condition this period will accept 12.0(12.0) of 20.0


290it [00:29,  9.85it/s]


71 well(s) accepted out of 290


100%|██████████| 71/71 [00:17<00:00,  4.03it/s]
